In [1]:
#aysenur
#experiment 1 de jupyter içerisinde weight update edilmediğini farkettim
#detayını gitmedim ama gpu da accelerator kullanmakla alakalı o yüzden her şeyi .py scripte aldım
#scripte aldıklarım aşağıda sırasıyla verildi, bir kere denemesi yapıldı
#sonrasında belirlenen parametrelerle script çalıştırıldı
#CosineWarmupScheduler eklendi

In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers import TensorBoardLogger
import wandb

In [3]:
from DataPreparation import get_data

In [4]:
class TimeSeriesDataset(Dataset):
    def __init__(self, 
                 currency_list,
                 x: np.ndarray, 
                 y: np.ndarray,
                 data_use_type,
                 train_percentage,
                 val_percentage,
                 test_percentage,
                 seq_len, 
                 ):
        self.currencies = currency_list
        self.n_currencies = len(self.currencies)
        self.x = torch.tensor(x[:self.n_currencies]).float()
        self.y = torch.tensor(y[:self.n_currencies]).long()
        self.seq_len = seq_len
        self.data_use_type = data_use_type
        
        
        #self.train_size = int(len(self.x[0]) * train_percentage)
        self.val_size = int(len(self.x[0]) * val_percentage)
        self.test_size = int(len(self.x[0]) * test_percentage)
        self.train_size = len(self.x[0]) - self.val_size - self.test_size 
        
        self.train_mean = [self.x[i][:self.train_size].mean() for i in range(self.n_currencies)]
        self.train_std = [self.x[i][:self.train_size].std() for i in range(self.n_currencies)]
        
#         self.train_min = [self.x[i][:self.train_size].min() for i in range(n_currencies)]
#         self.train_max = [self.x[i][:self.train_size].max() for i in range(n_currencies)]
        
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - ( self.seq_len)

        elif self.data_use_type == "val":
            return self.val_size
  
        else:
            return self.test_size
        
    
    def __getitem__(self, index):
        
        item = dict()
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        for i in range(self.n_currencies):
            window = self.x[i][index:index+self.seq_len]
            window = (window -self.train_mean[i]) / self.train_std[i]
            
            item[self.currencies[i] + "_window"] = window
            item[self.currencies[i] + "_label"]  = self.y[i][index+self.seq_len]

        return item

In [5]:
class CosineWarmupScheduler(torch.optim.lr_scheduler._LRScheduler):
    
    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)
        
    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]
    
    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor

In [34]:
class LSTM_based_classification_model(pl.LightningModule):
    def __init__(self,
                 train_dataset,
                 val_dataset,
                 test_dataset,
                 calculate_loss_weights,
                 currencies,
                 num_classes,
                 window_size,
                 input_size,
                 batch_size,
                 lstm_hidden_sizes,
                 bidirectional,
                 learning_rate = 1e-3,
                 scheduler_step = 10,
                 scheduler_gamma = 0.1,
                 ):
        
        super().__init__()
        self.num_classes = num_classes
        self.currencies = currencies
        self.num_tasks = len(currencies)
        self.window_size = window_size
        self.input_size = input_size
        self.batch_size = batch_size
        
        self.lstm_hidden_sizes = lstm_hidden_sizes
        self.bidirectional = bidirectional 
        
        if calculate_loss_weights:
            loss_weights = []
            for i in range(self.num_tasks):
                train_labels = [int(train_dataset[n][self.currencies[i] +"_label"] )for n in range(train_dataset.__len__())]
                samples_size = pd.DataFrame({"label": train_labels}).groupby("label").size().to_numpy()
                loss_weights.append((1 / samples_size) * sum(samples_size)/2)
            self.weights = loss_weights
        else:
            self.weights = None
        
        self.lstm_1 = nn.LSTM(input_size = self.input_size, 
                              num_layers=1, 
                              batch_first=True, 
                              hidden_size = self.lstm_hidden_sizes[0], 
                              bidirectional = bidirectional)
        self.batch_norm1 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[0])
        
        if len(self.lstm_hidden_sizes) > 1:
            self.lstm_2 = nn.LSTM(input_size = self.lstm_hidden_sizes[0], 
                                  num_layers=1, 
                                  batch_first=True, 
                                  hidden_size = self.lstm_hidden_sizes[1], 
                                  bidirectional = bidirectional)
            self.batch_norm2 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[1])

            self.lstm_3 = nn.LSTM(input_size = self.lstm_hidden_sizes[1], 
                                  num_layers=1, 
                                  batch_first=True, 
                                  hidden_size = self.lstm_hidden_sizes[2], 
                                  bidirectional = bidirectional)
            self.batch_norm3 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[2])
        
        self.dropout = nn.Dropout(0.5)
        
        self.linear1 =[nn.Linear(self.lstm_hidden_sizes[-1], int(self.lstm_hidden_sizes[-1]/2))] * self.num_tasks
        self.linear1 = torch.nn.ModuleList(self.linear1)
        self.activation = nn.ReLU()
        
        self.output_layers = [nn.Linear(int(self.lstm_hidden_sizes[-1]/2), self.num_classes)] * self.num_tasks
        self.output_layers = torch.nn.ModuleList(self.output_layers)
        
        if self.weights != None:
            self.cross_entropy_loss = [nn.CrossEntropyLoss(weight= torch.tensor(weights).float()) for weights in self.weights]
        else:
            self.cross_entropy_loss = [nn.CrossEntropyLoss() for _ in range(self.num_tasks)]
        
        self.cross_entropy_loss = torch.nn.ModuleList(self.cross_entropy_loss)
        
        self.f1_score = pl.metrics.F1(num_classes=self.num_classes, average="macro")
        self.accuracy_score = pl.metrics.Accuracy()
        
        self.train_dl = DataLoader(train_dataset, batch_size=self.batch_size, shuffle = True)
        self.val_dl = DataLoader(val_dataset, batch_size=self.batch_size)
        self.test_dl = DataLoader(test_dataset, batch_size=self.batch_size)
        
        self.learning_rate = learning_rate
        self.scheduler_step = scheduler_step
        self.scheduler_gamma = scheduler_gamma
        
    def forward(self, x, i):

        batch_size = x.size()[0]
        
        x = x.view(batch_size, self.window_size, self.input_size) #(batch, window_len, feature_size)
        x, _  = self.lstm_1(x)
        
        x = self.dropout(x)

        x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm1(x.unsqueeze(0))
        
        if len(self.lstm_hidden_sizes) > 1:
            
            x = x.view(batch_size, self.window_size, x.size()[1])
            x, _  = self.lstm_2(x)

            x = self.dropout(x)

            x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
            x = self.batch_norm2(x.unsqueeze(0))

            x = x.view(batch_size, self.window_size, x.size()[1])
            x, _  = self.lstm_3(x)

            x = self.dropout(x)

            x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
            x = self.batch_norm3(x.unsqueeze(0))
        
        x = x.view(batch_size, self.window_size, x.size()[1])
        x = x[:, -1, :] # equivalent to return sequence = False on keras :)
        
        x = self.dropout(x)
        
        x = self.linear1[i](x)
        x = self.activation(x)
                 
        output = self.output_layers[i](x)
    
        return output
    
    
    def training_step(self, batch, batch_nb):
        
        loss = (torch.tensor(0.0, device="cuda:0", requires_grad=True) + \
                torch.tensor(0.0, device="cuda:0", requires_grad=True)) 
        # araştırılabilir
        for i in range(self.num_tasks):
            x, y = batch[self.currencies[i] + "_window"], batch[self.currencies[i] + "_label"]

            output = self.forward(x, i)
            #loss = F.nll_loss(output, y)
            loss += self.cross_entropy_loss[i](output, y)
            
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_train_acc', acc, on_epoch=True, prog_bar=True)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_train_f1', f1, on_epoch=True, prog_bar=True)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        
        return loss 
    
    def validation_step(self, batch, batch_nb):
        loss = torch.tensor(0.0, device="cuda:0") + torch.tensor(0.0, device="cuda:0")
        
        for i in range(self.num_tasks):
            x, y = batch[self.currencies[i] + "_window"], batch[self.currencies[i] + "_label"]

            output = self(x, i)
            #loss = F.nll_loss(output, y)
            loss += self.cross_entropy_loss[i](output, y)
 
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_val_acc', acc, on_epoch=True, prog_bar=True, reduce_fx=torch.mean)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_val_f1', f1, on_epoch=True, prog_bar=True, reduce_fx=torch.mean)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
    
    def test_step(self, batch, batch_nb):
        loss = torch.tensor(0.0, device="cuda:0") + torch.tensor(0.0, device="cuda:0")
        
        for i in range(self.num_tasks):
            x, y = batch[ self.currencies[i] + "_window"], batch[self.currencies[i] + "_label"]

            output = self(x, i)
#             print(y, torch.max(output, dim=1)[1])
#             print(F.softmax(output)) # mantıken fark etmiyor
            loss += self.cross_entropy_loss[i](output, y)
            
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_test_acc', acc, on_epoch=True, reduce_fx=torch.mean)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_test_f1', f1, on_epoch=True, reduce_fx=torch.mean)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('test_loss', loss, on_epoch=True, reduce_fx=torch.mean)

        
    def configure_optimizers(self):
        
        optimizer = torch.optim.AdamW(model.parameters(), lr= self.learning_rate)#AdamW does weight decay
#         scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
#                                                     step_size=self.scheduler_step, 
#                                                     gamma=self.scheduler_gamma)
        
        self.lr_scheduler = CosineWarmupScheduler(optimizer, 
                                                  warmup=50, 
                                                  max_iters=150* self.train_dl.__len__())
        return [optimizer]#, [{"scheduler": scheduler}]
    
    def optimizer_step(self, *args, **kwargs):
        super().optimizer_step(*args, **kwargs)
        self.lr_scheduler.step() # Step per iteration
    
    def train_dataloader(self):
        return self.train_dl

    def val_dataloader(self):
        return self.val_dl

    def test_dataloader(self):
        return self.test_dl

In [6]:
def name_model(config):
    task = "multi_task_" + "_".join(config["currency_list"]) if len(config["currency_list"]) > 1 else "single_task_" + config["currency_list"][0]
    classification = "multi_classification" if config["n_classes"] > 2 else "binary_classification"
    lstm = "stack_lstm" if len(config["lstm_hidden_sizes"]) > 1 else "single_lstm"
    trend_removed = "trend_removed" if config["remove_trend"] else ""
    loss_weighted = "loss_weighted" if config["loss_weight_calculate"] else ""

    return "_".join([task, lstm, loss_weighted, classification, trend_removed])

In [ ]:
#deneme 1

In [1]:
CONFIG = {#fix for this project
          "window_size": 50, 
          "dataset_percentages": [0.97, 0.007, 0.023],
          "frenquency": "D", 
          "neutral_quantile": 0.33,
          "batch_size": 16,
          "bidirectional": False}

In [2]:
config = CONFIG.copy()
config.update({"n_classes": 2,
          "currency_list": ['BTC'],
          "remove_trend": True,
          "lstm_hidden_sizes": [128, 128, 128],
          "loss_weight_calculate": False})

In [38]:
MODEL_NAME = name_model(config)

CURRENCY_LST = config["currency_list"]
N_CLASSES = config["n_classes"]
LSTM_HIDDEN_SIZES = config["lstm_hidden_sizes"]
BIDIRECTIONAL = config["bidirectional"]
REMOVE_TREND =config["remove_trend"]
LOSS_WEIGHT_CALCULATE = config["loss_weight_calculate"]

TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = config["dataset_percentages"] 
WINDOW_SIZE = config["window_size"]
FREQUENCY = config["frenquency"]
NEUTRAL_QUANTILE = config["neutral_quantile"] if N_CLASSES > 2 else 0 
BATCH_SIZE= config["batch_size"]
#####

X, y, features, dfs = get_data(CURRENCY_LST,
                            N_CLASSES,
                             FREQUENCY, 
                             WINDOW_SIZE,
                             neutral_quantile = NEUTRAL_QUANTILE,
                             log_price=True,
                             remove_trend=REMOVE_TREND,
                             include_indicators = False,
                             include_imfs = False
                            )
INPUT_FEATURE_SIZE = X.shape[-1]

train_dataset, val_dataset, test_dataset = [TimeSeriesDataset(CURRENCY_LST, 
                                                          X, 
                                                          y, 
                                                          dtype, 
                                                          TRAIN_PERCENTAGE, 
                                                          VAL_PERCENTAGE, 
                                                          TEST_PERCENTAGE, 
                                                          WINDOW_SIZE) for dtype in ['train', 'val', 'test']]

config["dataset_sizes"] = [len(train_dataset), len(val_dataset), len(test_dataset)]
####
wandb.init(project="price_change_2",
           config=config,
           name = MODEL_NAME)
logger = WandbLogger()
#     #logger = TensorBoardLogger("../output/models/lstm_model_logs", name="lstm_multi_task")

model = LSTM_based_classification_model(
    train_dataset = train_dataset,
     val_dataset = val_dataset,
     test_dataset = test_dataset,
     calculate_loss_weights = LOSS_WEIGHT_CALCULATE,
     currencies = CURRENCY_LST,
     num_classes = N_CLASSES,
     window_size = WINDOW_SIZE,
     input_size = INPUT_FEATURE_SIZE,
     batch_size=BATCH_SIZE,
     lstm_hidden_sizes = LSTM_HIDDEN_SIZES,
     bidirectional = BIDIRECTIONAL)

early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.003,
   patience=20,
   verbose=True,
   mode='min'
)

trainer = pl.Trainer(gpus=-1, 
                     max_epochs= 150,
                     logger = logger, 
                     callbacks=[early_stop_callback])
trainer.fit(model)

trainer.test()
wandb.finish()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type        | Params
----------------------------------------------------
0  | lstm_1             | LSTM        | 67.1 K
1  | batch_norm1        | BatchNorm2d | 256   
2  | lstm_2             | LSTM        | 132 K 
3  | batch_norm2        | BatchNorm2d | 256   
4  | lstm_3             | LSTM        | 132 K 
5  | batch_norm3        | BatchNorm2d | 256   
6  | dropout            | Dropout     | 0     
7  | linear1            | ModuleList  | 8.3 K 
8  | activation         | ReLU        | 0     
9  | output_layers      | ModuleList  | 130   
10 | cross_entropy_loss | ModuleList  | 0     
11 | f1_score           | F1          | 0     
12 | accuracy_score     | Accuracy    | 0     
----------------------------------------------------
340 K     Trainable params
0         Non-trainable params
340 K     Total params
1.362     Total estimated model params 

/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Metric val_loss improved. New best score: 0.664


Metric val_loss improved by 0.082 >= min_delta = 0.003. New best score: 0.582


Metric val_loss improved by 0.135 >= min_delta = 0.003. New best score: 0.447


/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'BTC_test_acc': 0.6774193644523621,
 'BTC_test_f1': 0.6693548560142517,
 'test_loss': 0.6429551839828491}
--------------------------------------------------------------------------------


BTC_train_acc_step,0.6875
BTC_train_f1_step,0.67611
train_loss_step,0.50698
epoch,19
trainer/global_step,1557
_runtime,61
_timestamp,1621455965
_step,69
BTC_train_acc_epoch,0.71576
BTC_train_f1_epoch,0.69879
train_loss_epoch,0.57761


BTC_train_acc_step,▃▁▃▅▅▅▇▄▅▅█▅▃▃▁▆▄▅▇▅▆▃▅▃▅▅▅▅▅▅▅
BTC_train_f1_step,▂▁▃▅▅▅▇▂▄▅█▄▃▃▁▆▃▅▇▅▆▂▄▃▅▄▅▄▄▄▄
train_loss_step,▅▆▅▄▅▂▁▄▄▄▁▄▇▆█▂▆▄▁▃▃▄▅▅▃▄▃▅▄▅▃
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
BTC_train_acc_epoch,▁▃▆▇▇▇▇▇▇███▇█████▇
BTC_train_f1_epoch,▁▃▆▇▇▇▇▇▇███▇▇████▇
train_loss_epoch,█▆▃▂▃▂▂▂▂▂▁▁▂▂▂▂▁▁▁


In [1]:
#denemeleri tamamlandı experiment_lstm.py dosyasına yazıldı işlemler
def experiment(script):
    !python ../pipelines/multi_task_price_change_prediction/experiment_lstm.py $script

In [2]:
CONFIG = {#fix for this project
          "window_size": 50, 
          "dataset_percentages": [0.97, 0.007, 0.023],
          "frenquency": "D", 
          "neutral_quantile": 0.33,
          "batch_size": 16,
          "bidirectional": False}

In [3]:
from sklearn.model_selection import ParameterGrid
param_grid = {"n_classes": [2,3],
          "currency_list": [['BTC'], ['ETH'], ['LTC'], ['BTC', 'ETH'],  ['BTC', 'ETH', 'LTC']],
          "remove_trend": [True, False],
          "lstm_hidden_sizes": [[128], [128, 128, 128]],
          "loss_weight_calculate": [True, False]}

In [4]:
for c in (ParameterGrid(param_grid)):
    config = CONFIG.copy()
    config.update(c)
    script = "--currency-list " + " ".join([i for i in c["currency_list"]])
    script += " --lstm-list " + " ".join([str(i) for i in c["lstm_hidden_sizes"]])
    script += " -trend " + str(1 if c["remove_trend"] else 0)
    script += " -classes " + str(c["n_classes"] )
    script += " -weight " + str(1 if c["loss_weight_calculate"] else 0) 

    experiment(script)

wandb: Currently logged in as: aysenurk (use `wandb login --relogin` to force relogin)
2021-05-20 00:55:10.435653: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.10.30
wandb: Syncing run single_task_BTC_single_lstm_loss_weighted_binary_classification_trend_removed
wandb: ⭐️ View project at https://wandb.ai/aysenurk/price_change_2
wandb: 🚀 View run at https://wandb.ai/aysenurk/price_change_2/runs/1eprgmz4
wandb: Run data is saved locally in /home/aysenurk/Projects/OzU/CS_540_MLF/multi_task_price_change_prediction/notebooks/wandb/run-20210520_005508-1eprgmz4
wandb: Run `wandb offline` to turn off syncing.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type        | Params
---------------------------------------------------
0 | lstm_1             | LSTM        | 67.1 K
1 | batch_norm1 

wandb: Currently logged in as: aysenurk (use `wandb login --relogin` to force relogin)
2021-05-20 00:56:32.154796: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.10.30
wandb: Syncing run single_task_BTC_single_lstm_loss_weighted_binary_classification_
wandb: ⭐️ View project at https://wandb.ai/aysenurk/price_change_2
wandb: 🚀 View run at https://wandb.ai/aysenurk/price_change_2/runs/34r9vs22
wandb: Run data is saved locally in /home/aysenurk/Projects/OzU/CS_540_MLF/multi_task_price_change_prediction/notebooks/wandb/run-20210520_005630-34r9vs22
wandb: Run `wandb offline` to turn off syncing.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type        | Params
---------------------------------------------------
0 | lstm_1             | LSTM        | 67.1 K
1 | batch_norm1        | Batc

Epoch 25: 100%|█| 81/81 [00:01<00:00, 76.89it/s, loss=0.692, v_num=vs22, BTC_val
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
Testing: 100%|████████████████████████████████████| 2/2 [00:00<00:00, 98.63it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'BTC_test_acc': 0.5806451439857483,
 'BTC_test_f1': 0.36725807189941406,
 'test_loss': 0.6884135603904724}
--------------------------------------------------------------------------------

wandb: Waiting for W&B process to finish, PID 63051
wandb: Program ended successfully.
wandb:         

Testing: 100%|███████████████████████████████████| 2/2 [00:00<00:00, 118.86it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'BTC_test_acc': 0.4516128897666931,
 'BTC_test_f1': 0.40499287843704224,
 'test_loss': 0.9798224568367004}
--------------------------------------------------------------------------------

wandb: Waiting for W&B process to finish, PID 63252
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: /home/aysenurk/Projects/OzU/CS_540_MLF/multi_task_price_change_prediction/notebooks/wandb/run-20210520_005720-2ae5c60y/logs/debug.log
wandb: Find internal logs for this run at: /home/aysenurk/Projects/OzU/CS_540_MLF/multi_task_price_change_prediction/notebooks/wandb/run-20210520_005720-2ae5c60y/logs/debug-internal.log
wandb: Run summary:
wandb:    BTC_train_acc_step 0.625
wandb:     BTC_train_f1_step 0.613

Epoch 9:  99%|▉| 80/81 [00:00<00:00, 87.09it/s, loss=1.1, v_num=l2tj, BTC_val_ac
Validating: 0it [00:00, ?it/s]
Epoch 9: 100%|█| 81/81 [00:00<00:00, 84.18it/s, loss=1.1, v_num=l2tj, BTC_val_ac
Epoch 10:  99%|▉| 80/81 [00:01<00:00, 79.60it/s, loss=1.09, v_num=l2tj, BTC_val_
Validating: 0it [00:00, ?it/s]
Epoch 10: 100%|█| 81/81 [00:01<00:00, 77.27it/s, loss=1.09, v_num=l2tj, BTC_val_
Epoch 11:  99%|▉| 80/81 [00:00<00:00, 87.03it/s, loss=1.1, v_num=l2tj, BTC_val_a
Validating: 0it [00:00, ?it/s]
Epoch 11: 100%|█| 81/81 [00:00<00:00, 84.85it/s, loss=1.1, v_num=l2tj, BTC_val_a
Epoch 12:  99%|▉| 80/81 [00:00<00:00, 92.35it/s, loss=1.1, v_num=l2tj, BTC_val_a
Validating: 0it [00:00, ?it/s]
Epoch 12: 100%|█| 81/81 [00:00<00:00, 89.88it/s, loss=1.1, v_num=l2tj, BTC_val_a
                                                                                Metric val_loss improved by 0.018 >= min_delta = 0.003. New best score: 1.119
Epoch 13:  99%|▉| 80/81 [00:00<00:00, 84.05it/s, loss=1.09, v_num=l2tj

...

Epoch 0:  99%|▉| 72/73 [00:04<00:00, 14.57it/s, loss=0.683, v_num=25vq, BTC_val_
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Metric val_loss improved. New best score: 0.691
Epoch 0: 100%|█| 73/73 [00:05<00:00, 14.54it/s, loss=0.683, v_num=25vq, BTC_val_
Epoch 1:  99%|▉| 72/73 [00:05<00:00, 13.96it/s, loss=0.617, v_num=25vq, BTC_val_
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Metric val_loss improved by 0.068 >= min_delta = 0.003. New best score: 0.623
Epoch 1: 100%|█| 73/73 [00:05<00:00, 13.90it/s, loss=0.617, v_num=25vq, BTC_val_
Epoch 2:  99%|▉| 72/73 [00:05<00:00, 14.28it/s, loss=0.563, v_num=25vq, BTC_val_
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Metric val_loss improved by 0.016 >= min_delta = 0.003. New best score: 0.608
Epoch 2: 100%|█| 73/73 [00:05<00:00, 14.23it/s, loss=0.563

wandb: Currently logged in as: aysenurk (use `wandb login --relogin` to force relogin)
2021-05-20 02:57:40.574165: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.10.30
wandb: Syncing run multi_task_BTC_ETH_LTC_stack_lstm__binary_classification_
wandb: ⭐️ View project at https://wandb.ai/aysenurk/price_change_2
wandb: 🚀 View run at https://wandb.ai/aysenurk/price_change_2/runs/1o3x1f2w
wandb: Run data is saved locally in /home/aysenurk/Projects/OzU/CS_540_MLF/multi_task_price_change_prediction/notebooks/wandb/run-20210520_025739-1o3x1f2w
wandb: Run `wandb offline` to turn off syncing.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type        | Params
----------------------------------------------------
0  | lstm_1             | LSTM        | 67.1 K
1  | batch_norm1        | BatchNo

Testing: 100%|████████████████████████████████████| 2/2 [00:00<00:00, 24.68it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'BTC_test_acc': 0.3928571343421936,
 'BTC_test_f1': 0.2789115607738495,
 'ETH_test_acc': 0.6428571343421936,
 'ETH_test_f1': 0.38938775658607483,
 'LTC_test_acc': 0.4285714328289032,
 'LTC_test_f1': 0.293949156999588,
 'test_loss': 0.7038599848747253}
--------------------------------------------------------------------------------

wandb: Waiting for W&B process to finish, PID 82654
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: /home/aysenurk/Projects/OzU/CS_540_MLF/multi_task_price_change_prediction/notebooks/wandb/run-20210520_025739-1o3x1f2w/logs/debug.log
wandb: Find internal logs for this run at: /home/aysenurk/Projects/OzU/CS_540_MLF/multi_task_price_change_prediction/notebooks/wan

Epoch 36: 100%|█| 73/73 [00:06<00:00, 11.31it/s, loss=0.903, v_num=2tag, BTC_val
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
Testing: 100%|████████████████████████████████████| 2/2 [00:00<00:00, 28.05it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'BTC_test_acc': 0.5357142686843872,
 'BTC_test_f1': 0.2323809713125229,
 'ETH_test_acc': 0.5,
 'ETH_test_f1': 0.2212051898241043,
 'LTC_test_acc': 0.4285714328289032,
 'LTC_test_f1': 0.1999756544828415,
 'test_loss': 0.9154524207115173}
------------------------------------------------------

Validation sanity check:   0%|                            | 0/1 [00:00<?, ?it/s]/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
Epoch 0:  99%|▉| 73/74 [00:04<00:00, 15.04it/s, loss=1.12, v_num=nx2r, BTC_val_a
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|█| 74/74 [00:04<00:00, 15.00it/s, loss=1.12, v_num=nx2r, BTC_val_a
                                                                                Metric val_loss improved. New best score: 1.066
Epoch 1: 100%|█| 74/74 [00:05<00:00, 14.65it/s, loss=1.1, v_num=nx2r, BTC_val_ac
Validating: 0it [00:00, ?it/s]
Epoch 1: 100%|█| 74/74 [00:05<00:00, 14.57it/s, loss=1.1, v_num=nx2r, BTC_val_ac
Epoch 2: 1